In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

from pandas import read_csv
from pandas import concat
from pandas import DataFrame

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, Dropout
from keras.layers import LSTM
from keras.utils import plot_model
from sklearn.metrics import mean_squared_error
from keras.models import load_model
from scipy import stats
import csv as csv

import copy

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
fileName = '1a.csv'
modelbestName = "mlp_1a_best_result_test"
data = read_csv(fileName)
dataAvg = [503.62859]
dataStd = [100.4557059]

nDiff = 1

def difference(data):
        return [data[i] - data[i - nDiff] for i in range(nDiff, len(data))]

def shiftData(data, lag, nOut = 1):
        dataOfOneColumn = DataFrame(data)
        dataOfOneColumnWithLag = list()
        for i in range(lag, 0, -1):
                dataOfOneColumnWithLag.append(dataOfOneColumn.shift(i))
        for i in range(0, nOut):
                dataOfOneColumnWithLag.append(dataOfOneColumn.shift(-i))
        dataOfOneColumnWithLag = concat(dataOfOneColumnWithLag, axis = 1)
        dataOfOneColumnWithLag.dropna(inplace = True)
        return dataOfOneColumnWithLag

def prepareDataset(data, lag, numPoints):
        trainingData = list()
        flag = True
        for column in data.columns:
              dataOfOneColumnWithLag = np.array(shiftData(data[column], lag))[:numPoints]
              if flag == True:
                        trainingData = dataOfOneColumnWithLag
                        flag = False
                        continue
              trainingData = np.append(trainingData, dataOfOneColumnWithLag, axis = 0)
        return trainingData

def train(trainingData, config):
        inputShape, numNode, numEpoch, batchSize = config
        if nDiff > 0:
                trainingData = np.array(difference(trainingData))
        trainX, trainY = trainingData[:, :-1], trainingData[:, -1]
        #trainX = trainX.reshape((trainX.shape[0], trainX.shape[1], 1))
        model = Sequential()
        #model.add(LSTM(numNode, activation = 'relu', input_shape = (inputShape, 1)))
        model.add(Dense(numNode, activation = 'relu'))
        #model.add(Dense(numNode, activation = 'relu'))
        #model.add(Dense(numNode, activation = 'relu'))
        #model.add(Dense(numNode, activation = 'relu'))
        #model.add(Dropout(0.2))
        model.add(Dense(1))
        model.compile(loss = 'mse' , optimizer = 'adam')
        history = model.fit(trainX, trainY, epochs = numEpoch, batch_size = batchSize, verbose = 0, shuffle = False)
        # summarize history for loss
        #plt.plot(history.history['loss'])
        #plt.plot(history.history['val_loss'])
        #plt.title('model loss')
        #plt.ylabel('loss')
        #plt.xlabel('epoch')
        #plt.legend(['train', 'test'], loc='upper left')
        #plt.show()
        #plt.close()
        #plt.savefig(modelName + "ind")
        #plot_model(model, to_file=modelName+'_.png', show_shapes=True, show_layer_names=True)  
        #print("training loss:" , np.mean(np.array(history.history['loss'])))
        #print("Validation loss:" , np.mean(np.array(history.history['val_loss'])))
        return model,np.mean(np.array(history.history['loss']))

def predict(model, testX, inputShape):
        testX = np.array(testX).reshape((1, inputShape))
        predictions = model.predict(testX, verbose = 0)
        return predictions[0]

def rsquared(x, y):
    """ Return R^2 where x and y are array-like."""

    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    return r_value**2

def measureRmse(actual, predicted, dataAvg, dataStd, columnName):
        actual = np.array(actual)*dataStd + dataAvg
        predicted = np.array(predicted)*dataStd + dataAvg
        #print(predicted[0],actual[1])
        #np.savetxt(modelName + "predicted_" + columnName, predicted)
        #np.savetxt("actual_" + columnName, actual)
        actualAvg10 = list()
        predictedAvg10 = list()
        for i in range(0, actual.shape[0], 7):
                actualAvg10.append(np.sum(actual[i:i+7]))
       	        predictedAvg10.append(np.sum(predicted[i:i+7]))
        #plt.plot(np.array(actualAvg10))
        #plt.plot(np.array(predictedAvg10))
        #plt.show()
        rmse=np.sqrt(mean_squared_error(np.array(actualAvg10), np.array(predictedAvg10)))
        r_squared=rsquared(np.array(actualAvg10),np.array(predictedAvg10))
        print("RMSE and 1-rsquared", rmse,1-r_squared)
        return rmse,r_squared
        #r_squared=rsquared(actual.reshape(actual.shape[0]), predicted.reshape(predicted.shape[0]))
        #return rmse
    
def train_predict(model,trainingData,found):
        cnt = 0
        for column in data.columns:
            predictions = list()
            testData = np.array(shiftData(data[column][:trainingDataSize+3], lag))
            correction = testData[:, -1]
            if nDiff > 0:
                    testData = np.array(difference(testData))
            testX, testY = testData[:, :-1], testData[:, -1]
            for i in range(testX.shape[0]):
                predictedVal = predict(model, testX[i], lag)
                if nDiff > 0:
                    predictedVal = predictedVal + correction[i]
                    testY[i] = testY[i] + correction[i]
                    predictions.append(predictedVal)
            t_error,t_rsq = measureRmse(testY, predictions, dataAvg[cnt], dataStd[cnt], column)
            cnt = cnt + 1
        print(found)
        if found==1:
            np.savetxt("/home/tonystark/shruti_work/genetic-algorithm-master/genetic-algorithm-master/GAwithbatchsize/a_noshuffle_nodrop/"+str(modelbestName)+"_training_result", predictions)
        return t_error,t_rsq

def test(model, data, config, testDataFirstPredictionIndex,found):
        cnt = 0
        totalError = 0
        lag = config[0]
        testDataStartingIndex = testDataFirstPredictionIndex - lag
        for column in data.columns:
                predictions = list()
                testData = np.array(shiftData(data[column][testDataStartingIndex-1:], lag))
                correction = testData[:, -1]
                if nDiff > 0:
                        testData = np.array(difference(testData))
                testX, testY = testData[:, :-1], testData[:, -1]
                for i in range(testX.shape[0]):
                        predictedVal = predict(model, testX[i], lag)
                        if nDiff > 0:
                                predictedVal = predictedVal + correction[i]
                                testY[i] = testY[i] + correction[i]
                        predictions.append(predictedVal)
                error,rsq = measureRmse(testY, predictions, dataAvg[cnt], dataStd[cnt], column)
                totalError += error
                cnt = cnt + 1
        if found==1:
            np.savetxt("/home/tonystark/shruti_work/genetic-algorithm-master/genetic-algorithm-master/GAwithbatchsize/a_noshuffle_nodrop/"+str(modelbestName)+"_test_result", predictions)
        return totalError,rsq

def executeModel(data, config, numPoints,found):
    f=open("/home/tonystark/shruti_work/genetic-algorithm-master/genetic-algorithm-master/GAwithbatchsize/a_noshuffle_nodrop/result_1a.csv",'a')
    writer=csv.writer(f)
    trainingData = prepareDataset(data, lag, numPoints)
    bestError=30
    for j in range(30):
        model,train_loss = train(trainingData, config)
        t_error,t_rsq=train_predict(model,trainingData,found)
        error,rsq = test(model, data, config, numPoints,found)
        #print(error)
        modelName="Model_1a_"+str(j)+"_"+str(error)
        model.save("/home/tonystark/shruti_work/genetic-algorithm-master/genetic-algorithm-master/GAwithbatchsize/a_noshuffle_nodrop/"+
modelName)
        writer.writerow([modelName,t_error,t_rsq,error,rsq])
        if error < bestError:
            bestError = error
            #bestmodelName="best_model"
            model.save("/home/tonystark/shruti_work/genetic-algorithm-master/genetic-algorithm-master/GAwithbatchsize/a_noshuffle_nodrop/"+
modelbestName)
    return bestError

found=0
lag = 2
trainingDataSize = 1306
config = [lag, 8, 8, 20]
trainingData = prepareDataset(data, lag,trainingDataSize)
print(executeModel(data, config, trainingDataSize,found))

model = load_model("/home/tonystark/shruti_work/genetic-algorithm-master/genetic-algorithm-master/GAwithbatchsize/a_noshuffle_nodrop/"+modelbestName)
found=1
test(model, data, config, trainingDataSize,found)
train_predict(model,trainingData,found)







In [1]:
import os
os.getcwd()

'C:\\Users\\user'